In [1]:
 # Azure Blob Storage access info
 blob_account_name = "azureopendatastorage"
 blob_container_name = "nyctlc"
 blob_relative_path = "yellow"
    
 # Construct connection path
 wasbs_path = f'wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/{blob_relative_path}'
 print(wasbs_path)
    
 # Read parquet data from Azure Blob Storage path
 blob_df = spark.read.parquet(wasbs_path)

StatementMeta(, 9056ff2f-8cc5-48a8-a5e2-f0e45a51b055, 3, Finished, Available)

wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow


In [3]:
     # Declare file name    
     file_name = "yellow_taxi"
    
     # Construct destination path
     output_parquet_path = f"abfss://0f8a6bf2-e474-46f4-aa7b-4dad7187ed76@onelake.dfs.fabric.microsoft.com/943783e8-b897-4c76-8ee7-09301e06dc0b/Files/rawData/{file_name}"
     print(output_parquet_path)
        
     # Load the first 1000 rows as a Parquet file
     blob_df.limit(1000).write.mode("overwrite").parquet(output_parquet_path)

StatementMeta(, 9056ff2f-8cc5-48a8-a5e2-f0e45a51b055, 5, Finished, Available)

abfss://0f8a6bf2-e474-46f4-aa7b-4dad7187ed76@onelake.dfs.fabric.microsoft.com/943783e8-b897-4c76-8ee7-09301e06dc0b/Files/rawData/yellow_taxi


In [4]:
 from pyspark.sql.functions import col, to_timestamp, current_timestamp, year, month
    
 # Read the parquet data from the specified path
 raw_df = spark.read.parquet(output_parquet_path)   
    
 # Add dataload_datetime column with current timestamp
 filtered_df = raw_df.withColumn("dataload_datetime", current_timestamp())
    
 # Filter columns to exclude any NULL values in storeAndFwdFlag
 filtered_df = filtered_df.filter(raw_df["storeAndFwdFlag"].isNotNull())
    
 # Load the filtered data into a Delta table
 table_name = "yellow_taxi"  # Replace with your desired table name
 filtered_df.write.format("delta").mode("append").saveAsTable(table_name)
    
 # Display results
 display(filtered_df.limit(1))

StatementMeta(, 9056ff2f-8cc5-48a8-a5e2-f0e45a51b055, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 385e2cdb-1671-4bf6-9d33-c2afa5578394)

In [5]:
 from pyspark.sql.functions import col, to_timestamp, current_timestamp, year, month
 
 # Read the parquet data from the specified path
 raw_df = spark.read.parquet(output_parquet_path)    

 # Add dataload_datetime column with current timestamp
 opt_df = raw_df.withColumn("dataload_datetime", current_timestamp())
    
 # Filter columns to exclude any NULL values in storeAndFwdFlag
 opt_df = opt_df.filter(opt_df["storeAndFwdFlag"].isNotNull())
    
 # Enable V-Order
 spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
    
 # Enable automatic Delta optimized write
 spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
    
 # Load the filtered data into a Delta table
 table_name = "yellow_taxi_opt"  # New table name
 opt_df.write.format("delta").mode("append").saveAsTable(table_name)
    
 # Display results
 display(opt_df.limit(1))

StatementMeta(, 9056ff2f-8cc5-48a8-a5e2-f0e45a51b055, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, d0d8624a-8954-40d4-8dc5-96592ff6122b)

In [6]:
 # Load table into df
 delta_table_name = "yellow_taxi"
 table_df = spark.read.format("delta").table(delta_table_name)
    
 # Create temp SQL table
 table_df.createOrReplaceTempView("yellow_taxi_temp")
    
 # SQL Query
 table_df = spark.sql('SELECT * FROM yellow_taxi_temp')
    
 # Display 10 results
 display(table_df.limit(10))

StatementMeta(, 9056ff2f-8cc5-48a8-a5e2-f0e45a51b055, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, e3cd2a25-8957-4477-bb9d-c5c9c34a67ea)

In [7]:
 # Load table into df
 delta_table_name = "yellow_taxi_opt"
 opttable_df = spark.read.format("delta").table(delta_table_name)
    
 # Create temp SQL table
 opttable_df.createOrReplaceTempView("yellow_taxi_opt")
    
 # SQL Query to confirm
 opttable_df = spark.sql('SELECT * FROM yellow_taxi_opt')
    
 # Display results
 display(opttable_df.limit(10))

StatementMeta(, 9056ff2f-8cc5-48a8-a5e2-f0e45a51b055, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 574ab683-bab9-419b-b951-e0eee3568875)